In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!nvidia-smi


Sat Jan 18 09:40:10 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import json
import torch
from transformers import pipeline, set_seed
with open("test.json") as train_json:
    train = json.load(train_json)

with open("train.json") as test_json:
    test = json.load(test_json)

indix_label_dico = {}
for indix, key in enumerate(train):
    indix_label_dico[key] = indix

data = {'train': {'text':[], 'label':[]},
        'test': {'text':[], 'label':[]},
        'val': {'text':[], 'label':[]}}

for topic, corpus in train.items():
    for sentence in corpus:
      data['train']['text'].append(sentence)
      data['train']['label'].append(indix_label_dico[topic])

for topic, corpus in test.items():
    for sentence in corpus:
      data['test']['text'].append(sentence)
      data['test']['label'].append(indix_label_dico[topic])

In [4]:
from datasets import Dataset, DatasetDict
dataset = DatasetDict()
dataset_train = Dataset.from_dict(data['train'])
dataset_test= Dataset.from_dict(data['test'])

dataset_dict = DatasetDict({
    'train': dataset_train,
    'test':dataset_test})

In [13]:
from transformers import T5ForSequenceClassification, AutoTokenizer, AdamW
import torch

# Charger le modèle et le tokenizer
model_name = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForSequenceClassification.from_pretrained(model_name, num_labels=12)
model.to("cuda")

# Tokenisation
def fonc_tokenisation(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

inputs = dataset_dict.map(fonc_tokenisation, batched=True)
inputs = inputs.remove_columns(['text'])
inputs = inputs.rename_column("label", "labels")

inputs.set_format("torch")


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1338 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [14]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(inputs['train'], shuffle = True, batch_size=8)
test_dataloader = DataLoader(inputs['test'], batch_size=8)

# Optimiseur
optimizer = AdamW(model.parameters(), lr=5e-4)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# Boucle d'entraînement
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to("cuda") for k, v in batch.items()}  # Déplacer les données sur le GPU
        print(batch["labels"])
        outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"], labels=batch["labels"])
        logits = outputs.logits
        loss = loss_fn(logits, batch["labels"])
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



tensor([ 8,  2,  8,  1,  9,  6, 11,  0], device='cuda:0')
Epoch 1, Loss: 2.4777607917785645
tensor([ 5,  0,  3, 11, 10,  9,  1, 10], device='cuda:0')
Epoch 1, Loss: 2.7180428504943848
tensor([ 7,  0,  6,  4,  0,  3,  7, 11], device='cuda:0')
Epoch 1, Loss: 2.57663631439209
tensor([ 5,  2, 11, 11,  8, 11,  3, 10], device='cuda:0')
Epoch 1, Loss: 2.5587031841278076
tensor([4, 5, 8, 4, 9, 0, 8, 8], device='cuda:0')
Epoch 1, Loss: 2.508005142211914
tensor([ 6,  3,  9, 10,  6,  7,  1,  6], device='cuda:0')
Epoch 1, Loss: 2.471616268157959
tensor([ 1,  3, 10,  9,  5, 10,  9,  0], device='cuda:0')
Epoch 1, Loss: 2.4668891429901123
tensor([ 5,  6,  4, 11,  7,  4,  3,  5], device='cuda:0')
Epoch 1, Loss: 2.531216859817505
tensor([ 5,  8,  4,  3,  7,  8, 11,  1], device='cuda:0')
Epoch 1, Loss: 2.47784161567688
tensor([ 4,  5, 10,  8,  1, 11,  8,  3], device='cuda:0')
Epoch 1, Loss: 2.3693792819976807
tensor([ 1,  9, 10,  6,  9,  7,  1,  3], device='cuda:0')
Epoch 1, Loss: 2.4172298908233643
ten

In [16]:
# Phase de test
model.eval()  # Mettre le modèle en mode évaluation
all_predictions = []
all_labels = []

with torch.no_grad():  # Désactiver le calcul des gradients
    for batch in test_dataloader:
        batch = {k: v.to("cuda") for k, v in batch.items()}  # Déplacer les données sur le GPU

        # Passe avant
        outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
        logits = outputs.logits

        # Obtenir les prédictions
        predictions = torch.argmax(logits, dim=-1)  # Classe avec le score le plus élevé
        all_predictions.extend(predictions.cpu().tolist())  # Stocker les prédictions
        all_labels.extend(batch["labels"].cpu().tolist())   # Stocker les labels réels

# Évaluation des performances
from sklearn.metrics import classification_report

print(classification_report(all_labels, all_predictions))


              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       0.75      1.00      0.86         3
           2       0.67      0.67      0.67         3
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         3
           7       1.00      0.67      0.80         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       1.00      0.67      0.80         3
          11       1.00      1.00      1.00         3

    accuracy                           0.92        36
   macro avg       0.93      0.92      0.92        36
weighted avg       0.93      0.92      0.92        36



In [ ]:
import json
with open("train.json") as train_json:
    train = json.load(train_json)


sentences = []
for _, corpus in train.items():
    for sentence in corpus:
      sentences.append(sentence)

In [12]:
generator = pipeline('text-generation', model='gpt2')
print(generator(f'this is a test', max_length=100, num_return_sequences=2))

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'this is a test," Dr. Joss said. The results confirm the notion that we do better when exposed to low dose sodium iodide (NaCl). However, further evidence that sodium chloride may prevent the formation of lead-based lead in the bones has yet to be presented in the laboratory or in a clinical trial.\n\n"There is a clear benefit for children with lead exposure," Joss said. "If there were enough information in this study and the results could be applied to the'}, {'generated_text': "this is a test-based script that runs in my web server and only outputs raw HTML.\n\nFor me, this is particularly helpful for things like custom CSS or the DOM, using Javascript. For a site owner using the same site layout all but the first time they are using different browsers with the same CSS, you'll be seeing this message from the Web Console. And as you can see it looks much better when you run it for a minute\n\nI ran this script twice after updating"}]


In [15]:
from transformers import pipeline, set_seed
generated_sentences = []
for sentence in sentences:
    generation = generator(f"Synonym sentences : {sentence},", max_length=len(sentence.split())*2, num_return_sequences=1)
    for gen in generation:
        generated_sentences.append(gen['generated_text'])
    print(generated_sentences[-5:-1])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[]


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Synonym sentences : The mayor announced a new initiative to improve public transportation., which, in turn,']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

['Synonym sentences : Serena Williams continues to dominate the tennis world with her powerful serve., and has become the queen of', 'Synonym sentences : The World Cup is the most prestigious tournament in international soccer., The most-viewed-', 'Synonym sentences : Artificial intelligence is changing the way we live and work., we humans are changing how', 'Synonym sentences : The latest iPhone has a number of exciting new features., App of the Year (']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Synonym sentences : The World Cup is the most prestigious tournament in international soccer., The most-viewed-', 'Synonym sentences : Artificial intelligence is changing the way we live and work., we humans are changing how', 'Synonym sentences : The latest iPhone has a number of exciting new features., App of the Year (', 'Synonym sentences : Cybersecurity is becoming increasingly important as more and more data moves online., We know that computer security is']


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Synonym sentences : Artificial intelligence is changing the way we live and work., we humans are changing how', 'Synonym sentences : The latest iPhone has a number of exciting new features., App of the Year (', 'Synonym sentences : Cybersecurity is becoming increasingly important as more and more data moves online., We know that computer security is', 'Synonym sentences : The new Marvel movie is breaking box office records., the most-anticipated']
['Synonym sentences : The latest iPhone has a number of exciting new features., App of the Year (', 'Synonym sentences : Cybersecurity is becoming increasingly important as more and more data moves online., We know that computer security is', 'Synonym sentences : The new Marvel movie is breaking box office records., the most-anticipated', 'Synonym sentences : The Grammy Awards are a celebration of the best music of the year., by Steve Cervantes (2011)']
